# Linear Regression

Forecast the demand of a product for a given week, at a particular store

In [1]:
%matplotlib inline
import sys
from __future__ import print_function
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
rng = np.random

Set the csv file name 

In [2]:
train_filename = "./dataset/train.csv"
test_filename = "./dataset/test.csv"

In [3]:
semanaKey = "Semana"
agenciaIdKey = "Agencia_ID"
canalIdKey = "Canal_ID"
rutaSakKey = "Ruta_SAK"
clienteIdKey = "Cliente_ID"
productIdKey = "Producto_ID"
ventaUniHoyKey = "Venta_uni_hoy"
ventaHoyKey = "Venta_hoy"
devUniProximaKey = "Dev_uni_proxima"
devProximaKey = "Dev_proxima"
demandaUniEquil = "Demanda_uni_equil"
idKey = "id"

In [4]:
train_columns = [semanaKey, agenciaIdKey, canalIdKey, rutaSakKey, clienteIdKey, productIdKey, ventaUniHoyKey, ventaHoyKey, devUniProximaKey, devProximaKey, demandaUniEquil]
test_columns = [idKey, semanaKey, agenciaIdKey, canalIdKey, rutaSakKey, clienteIdKey, productIdKey]

In [5]:
def get_train_data(product_id, client_id):
    df = pd.read_csv(train_filename, names= train_columns, delimiter=',', dtype={semanaKey: object}, skiprows=1)
    dfx = df.apply(pd.to_numeric, errors="ignore")
    train_data = dfx[(dfx.Producto_ID == product_id) & (dfx.Cliente_ID == client_id)]
    return train_data

def get_test_data(product_id, client_id):
    df = pd.read_csv(test_filename, names= test_columns, delimiter=',', dtype={semanaKey: object}, skiprows=1)
    dfx = df.apply(pd.to_numeric, errors="ignore")
    test_data = dfx[(dfx.Producto_ID == product_id) & (dfx.Cliente_ID == client_id)]
    return test_data

In [6]:
def print_division():
    print("")

def log_test_data(test_x, test_y):
    print_division()
    print("test_x")
    print("data = ",test_x)
    print("length = ",len(test_x))
    print("test_y")
    print("data = ",test_y)
    print("length = ",len(test_y))

def log_train_data(train_x, train_y):
    print_division()
    print("train_x")
    print("length = ",len(train_x))
    print("data = ",train_x)
    print("train y")
    print("length = ",len(train_y))
    print("data = ",train_y)

def log_cost_history(cost_history):
    print_division()
    print("Cost History")
    print("range = ",range(len(cost_history)))
    print("data = ",cost_history)

def log_predictions(pred_y):
    print_division()
    print("pred_y")
    print("length = ",len(pred_y))
    print("data = ",pred_y)

In [7]:
productId = 1240
clientId = 15766
print("Searching matches for Producto_ID = ",productId," & Cliente_ID = ",clientId)
train_data = get_train_data(productId, clientId)
test_data = get_test_data(productId, clientId)

Searching matches for Producto_ID =  1240  & Cliente_ID =  15766


IOError: File train.csv does not exist

In [ ]:
train_data_len = len(train_data)
test_data_len = len(test_data)

Parameters

In [ ]:
learning_rate = 0.01
training_epochs = 3000
display_step = 50

Training Data

In [ ]:
train_X = train_data[semanaKey].values
train_Y = train_data[ventaHoyKey].values
n_samples = train_X.shape[0]
log_train_data(train_X, train_Y)

Testing Data

In [ ]:
test_X = test_data[semanaKey].values
n_samples_test = test_X.shape[0]
test_Y = np.ones(n_samples_test)
log_test_data(test_X, test_Y)

Pure math

In [ ]:
mean = np.mean(train_Y)

tf Graph Input

In [ ]:
X = tf.placeholder("float")
Y = tf.placeholder("float")

Set model weights

In [ ]:
W = tf.Variable(rng.randn(), name="weight")
b = tf.Variable(rng.randn(), name="bias")

Construct a linear model

In [ ]:
pred = tf.add(tf.multiply(X, W), b)

Mean squared error

In [ ]:
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

Gradient descent
Note, minimize() knows to modify W and b because Variable objects are trainable=True by default

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

Initializing the variables

In [ ]:
init = tf.global_variables_initializer()
cost_history = []

In [ ]:
print("Starting Session...")

Launch the graph

In [ ]:
with tf.Session() as sess:
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y: y})
        
        c = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
        cost_history.append(c)

        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b))

    print("Optimization Finished!")
    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')
    
    # Graphic display

    # Linear regression figure
    f1 = plt.figure() 
    ax1 = f1.add_subplot(111)
    ax1.plot(train_X, train_Y, 'ro', label='Original data')
    ax1.plot(train_X, sess.run(W) * train_X + sess.run(b), label='Fitted line')
    ax1.legend()
    
    #Cost history figure
    f2 = plt.figure()
    ax2 = f2.add_subplot(111)
    ax2.plot(range(len(cost_history)),cost_history,color='r')
    ax2.axis([0,training_epochs,0,np.max(cost_history)])
    log_cost_history(cost_history) 

    # Predict data
    print("Predicting data...") 
    test_X = test_data[semanaKey].values
    pred_y = sess.run(pred, feed_dict={X: test_X})
    mse = tf.reduce_mean(tf.square(pred_y - test_Y))
    print("MSE: %.4f" % sess.run(mse)) 
    log_predictions(pred_y)

    '''
    # Testing example
    print("Testing... (Mean square loss Comparison)")
    testing_cost = sess.run(
        tf.reduce_sum(tf.pow(pred - Y, 2)) / (2 * test_X.shape[0]),
        feed_dict={X: test_X, Y: test_Y})  # same function as cost above
    print("Testing cost=", testing_cost)
    print("Absolute mean square loss difference:", abs(
        training_cost - testing_cost))
    '''

    #Prediction figure
    f3 = plt.figure()
    ax3 = f3.add_subplot(111)
    ax3.scatter(test_Y, pred_y)
    ax3.plot([test_Y.min(), test_Y.max()], [test_Y.min(), test_Y.max()], 'k--', lw=3)
    ax3.set_xlabel('Measured')
    ax3.set_ylabel('Predicted')
    plt.show()
    
    print("Closing Session...")
    sess.close()